# Classificador de Risco com TF-IDF
Este notebook treina um classificador simples para identificar frases de alto ou baixo risco com base em descrições de sintomas.

### Objetivos
- Carregar um conjunto de frases rotuladas com níveis de risco.
- Transformar o texto em vetores TF-IDF.
- Treinar um modelo de regressão logística e avaliar o desempenho.
- Testar o modelo com novas frases para validar o comportamento.

In [ ]:
from pathlib import Path

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
BASE_DIR = Path.cwd().parents[0]
data_path = BASE_DIR / "data" / "classificacao_risco.csv"

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

raw_df = pd.read_csv(data_path)
raw_df.head()

### Exploração Rápida
Verificamos o balanceamento da base simulada e, em seguida, identificamos as principais palavras para cada classe a partir dos coeficientes da regressão logística.

In [ ]:
raw_df['situacao'].value_counts()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    raw_df["frase"], raw_df["situacao"], test_size=0.3, random_state=42, stratify=raw_df["situacao"]
)

pipeline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(ngram_range=(1, 2), min_df=1)),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced")),
    ]
)

pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
print(f"Acurácia: {pipeline.score(X_test, y_test):.2f}")
print("\nRelatório de classificação:\n")
print(classification_report(y_test, y_pred))

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))

### Principais Termos por Classe
A regressão logística permite interpretar pesos dos termos. Valores positivos indicam associação com `alto risco`, enquanto negativos sugerem `baixo risco`.

In [ ]:
vectorizer = pipeline.named_steps['tfidf']
clf = pipeline.named_steps['clf']
feature_names = vectorizer.get_feature_names_out()
classes = list(clf.classes_)
if len(classes) != 2:
    raise ValueError('Esperado classificador binário para inspecionar pesos.')
positive_class = classes[1]
coef_positive = clf.coef_[0]
if positive_class.lower() == 'alto risco':
    coefs_alto = coef_positive
    coefs_baixo = -coef_positive
else:
    coefs_alto = -coef_positive
    coefs_baixo = coef_positive
top_alto = coefs_alto.argsort()[-10:][::-1]
top_baixo = coefs_baixo.argsort()[-10:][::-1]
print('Termos mais associados a ALTO risco:')
for idx in top_alto:
    print(f'  {feature_names[idx]} - peso {coefs_alto[idx]:.3f}')

print('Termos mais associados a BAIXO risco:')
for idx in top_baixo:
    print(f'  {feature_names[idx]} - peso {coefs_baixo[idx]:.3f}')


### Observações sobre Viés
Por se tratar de uma base sintética e pequena, a cobertura de vocabulário é limitada. Termos muito específicos foram rotulados manualmente; portanto, recomenda-se expandir a base antes de utilizar o modelo em produção.

In [ ]:
novas_frases = [
    "sinto dor no peito e falta de ar depois de subir escadas",
    "leve desconforto no ombro após alongamento",
    "visão turva, pressão alta e suor frio na madrugada",
]

predicoes = pipeline.predict(novas_frases)

for frase, risco in zip(novas_frases, predicoes):
    print(f"Frase: {frase}\n → Risco previsto: {risco}\n")